In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Function to plot the coverage per base of a genome with and without zoom
def coverage_graph(input, output, amp_start, amp_end, zoom_start=None, zoom_end=None, title=None, total_coverage=None):
    df = pd.read_csv(input, sep='\t', names=['genome','base', 'coverage'])

    sns.lineplot(data=df, x='base', y='coverage')
    plt.title(title + '\n' + 'Total coverage: ' + total_coverage)
    plt.xlabel('Nucleotide position')
    plt.ylabel('Coverage')
    plt.axvline(x=amp_start, color='red', linestyle='--')
    plt.axvline(x=amp_end, color='red', linestyle='--')
    plt.savefig(output)
    plt.savefig(output.replace('.png', '.svg'))
    plt.clf()

    if zoom_start and zoom_end:
        sns.lineplot(data=df, x='base', y='coverage')
        plt.xlim(zoom_start, zoom_end)
        plt.title(title + '\n' + 'Total coverage: ' + total_coverage)
        plt.xlabel('Nucleotide position')
        plt.ylabel('Coverage')
        plt.axvline(x=amp_start, color='red', linestyle='--')
        plt.axvline(x=amp_end, color='red', linestyle='--')
        plt.savefig(output.replace('.png', '_zoom.png'))
        plt.savefig(output.replace('.png', '_zoom.svg'))
        plt.clf()
        
# Execution of the function for each genome
# coverage_graph(input='./01_besthit_Caudoviricetes_sp._isolate_ctlNO7_viral_allsamples_aligned_genomecov.txt',
#                output='./01_besthit_Caudoviricetes_sp._isolate_ctlNO7_viral_allsamples_aligned_genomecov.png',
#                amp_start=116,
#                amp_end=463,
#             #    zoom_start=1,
#             #    zoom_end=10562,
#                title='Viral coverage: Caudoviricetes sp. isolate ctlNO7',
#                total_coverage='7.17%')

# coverage_graph(input='./02_besthit_Clostridiales_bacterium_isolate_KR001_HIC_0007_viral_allsamples_aligned_genomecov.txt',
#                output='./02_besthit_Clostridiales_bacterium_isolate_KR001_HIC_0007_viral_allsamples_aligned_genomecov.png',
#                amp_start=2213911,
#                amp_end=2214257,
#             #    zoom_start=2203894,
#             #    zoom_end=2224357,
#                title='Viral coverage: Clostridiales bacterium isolate KR001 HIC 0007',
#                total_coverage='94.46%')

# coverage_graph(input='./03_besthit_Caudoviricetes_sp._isolate_ctUyu4_viral_allsamples_aligned_genomecov.txt',
#                output='./03_besthit_Caudoviricetes_sp._isolate_ctUyu4_viral_allsamples_aligned_genomecov.png',
#                amp_start=14793,
#                amp_end=15140,
#             #    zoom_start=4776,
#             #    zoom_end=16000,
#                title='Viral coverage: Caudoviricetes sp. isolate ctUyu4',
#                total_coverage='18.48%')

# coverage_graph(input='./04_besthit_Caudoviricetes_sp._isolate_ctInE22_viral_allsamples_aligned_genomecov.txt',
#                output='./04_besthit_Caudoviricetes_sp._isolate_ctInE22_viral_allsamples_aligned_genomecov.png',
#                amp_start=117,
#                amp_end=462,
#             #    zoom_start=1,
#             #    zoom_end=10563,
#                title='Viral coverage: Caudoviricetes sp. isolate ctInE22',
#                total_coverage='41.87%')

# coverage_graph(input='./05_besthit_Caudoviricetes_sp._isolate_ctvmC4_viral_allsamples_aligned_genomecov.txt',
#                output='./05_besthit_Caudoviricetes_sp._isolate_ctvmC4_viral_allsamples_aligned_genomecov.png',
#                amp_start=117,
#                amp_end=462,
#             #    zoom_start=1,
#             #    zoom_end=10563,
#                title='Viral coverage: Caudoviricetes sp. isolate ctvmC4',
#                total_coverage='13.69%')



In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
# %pip install scipy
from scipy.stats import mannwhitneyu

# Boxplot function to plot number of reads per sample
def boxplot(input, title):
    df = pd.read_csv(input, sep='\t')

    # Add group column to the dataframe
    df['group'] = df['sample'].apply(lambda x: 'H' if 'H' in x else ('OMC' if 'OMC' in x else 'O'))

    # Remove the H group
    df = df[df['group'] != 'H']

    # Define colors for the boxplot
    # colors = ['blue', 'orange', 'red']
    colors = ['orange', 'red']
    # Boxplot with reads aligned per sample
    ax = sns.boxplot(data=df, x='group', y='reads', showfliers=False, palette=colors)
    sns.stripplot(data=df, x='group', y='reads', color='black', size=4)
    plt.title(title)
    plt.xlabel('')
    plt.ylabel('Reads aligned')

    # Perform mannwhitneyu analysis
    # group_H = df[df['group'] == 'H']['reads']
    group_OMC = df[df['group'] == 'OMC']['reads']
    group_O = df[df['group'] == 'O']['reads']

    # Compare group H and OMC
    # u, p_H_OMC = mannwhitneyu(group_H, group_OMC)

    # Compare group H and O
    # u, p_H_O = mannwhitneyu(group_H, group_O)

    # Compare group OMC and O
    u, p_OMC_O = mannwhitneyu(group_OMC, group_O)

    # Get the y-axis limits
    bottom, top = ax.get_ylim()
    y_range = top - bottom

    # Significance bars
    # significant_combinations = [((0, 2), p_H_OMC), ((0, 1), p_H_O), ((1, 2), p_OMC_O)]
    significant_combinations = [((0, 1), p_OMC_O)]
    for i, significant_combination in enumerate(significant_combinations):
        # Columns corresponding to the datasets of interest
        x1 = significant_combination[0][0]
        x2 = significant_combination[0][1]
        # What level is this bar among the bars above the plot?
        level = len(significant_combinations) - i
        # Plot the bar
        bar_height = (y_range * 0.07 * level) + top
        bar_tips = bar_height - (y_range * 0.02)
        plt.plot(
            [x1, x1, x2, x2],
            [bar_tips, bar_height, bar_height, bar_tips], lw=1, c='k'
        )
        # Significance level
        p = significant_combination[1]
        if p < 0.001:
            sig_symbol = '***'
        elif p < 0.01:
            sig_symbol = '**'
        elif p < 0.05:
            sig_symbol = '*'
        else:
            sig_symbol = 'n.s.'
        text_height = bar_height + (y_range * 0.01)
        plt.text((x1 + x2) * 0.5, text_height, sig_symbol, ha='center', va='bottom', c='k')

    output = os.path.splitext(input)[0] + '.png'
    plt.savefig(output)
    output = os.path.splitext(input)[0] + '.svg'
    plt.savefig(output)
    plt.clf()

    # Boxplot with relative abundance per sample
    ax = sns.boxplot(data=df, x='group', y='relative_abundance', showfliers=False, palette=colors)
    sns.stripplot(data=df, x='group', y='relative_abundance', color='black', size=4)
    plt.title(title)
    plt.xlabel('')
    plt.ylabel('Relative abundance (%)')

    # Perform mannwhitneyu analysis
    # group_H = df[df['group'] == 'H']['reads']
    group_OMC = df[df['group'] == 'OMC']['reads']
    group_O = df[df['group'] == 'O']['reads']

    # Compare group H and OMC
    # u, p_H_OMC = mannwhitneyu(group_H, group_OMC)

    # Compare group H and O
    # u, p_H_O = mannwhitneyu(group_H, group_O)

    # Compare group OMC and O
    u, p_OMC_O = mannwhitneyu(group_OMC, group_O)

    # Get the y-axis limits
    bottom, top = ax.get_ylim()
    y_range = top - bottom

    # Significance bars
    # significant_combinations = [((0, 2), p_H_OMC), ((0, 1), p_H_O), ((1, 2), p_OMC_O)]
    significant_combinations = [((0, 1), p_OMC_O)]
    for i, significant_combination in enumerate(significant_combinations):
        # Columns corresponding to the datasets of interest
        x1 = significant_combination[0][0]
        x2 = significant_combination[0][1]
        # What level is this bar among the bars above the plot?
        level = len(significant_combinations) - i
        # Plot the bar
        bar_height = (y_range * 0.07 * level) + top
        bar_tips = bar_height - (y_range * 0.02)
        plt.plot(
            [x1, x1, x2, x2],
            [bar_tips, bar_height, bar_height, bar_tips], lw=1, c='k'
        )
        # Significance level
        p = significant_combination[1]
        if p < 0.001:
            sig_symbol = '***'
        elif p < 0.01:
            sig_symbol = '**'
        elif p < 0.05:
            sig_symbol = '*'
        else:
            sig_symbol = 'n.s.'
        text_height = bar_height + (y_range * 0.01)
        plt.text((x1 + x2) * 0.5, text_height, sig_symbol, ha='center', va='bottom', c='k')

    output = os.path.splitext(input)[0] + '_relative_abundance.png'
    plt.savefig(output)
    output = os.path.splitext(input)[0] + '_relative_abundance.svg'
    plt.savefig(output)
    plt.clf()

    # Boxplot with log10 abundance per sample
    ax = sns.boxplot(data=df, x='group', y='log10', showfliers=False, palette=colors)
    sns.stripplot(data=df, x='group', y='log10', color='black', size=4)
    plt.title(title)
    plt.xlabel('')
    plt.ylabel('log10 abundance')

        # Perform mannwhitneyu analysis
    # group_H = df[df['group'] == 'H']['reads']
    group_OMC = df[df['group'] == 'OMC']['reads']
    group_O = df[df['group'] == 'O']['reads']

    # Compare group H and OMC
    # u, p_H_OMC = mannwhitneyu(group_H, group_OMC)

    # Compare group H and O
    # u, p_H_O = mannwhitneyu(group_H, group_O)

    # Compare group OMC and O
    u, p_OMC_O = mannwhitneyu(group_OMC, group_O)

    # Get the y-axis limits
    bottom, top = ax.get_ylim()
    y_range = top - bottom

    # Significance bars
    # significant_combinations = [((0, 2), p_H_OMC), ((0, 1), p_H_O), ((1, 2), p_OMC_O)]
    significant_combinations = [((0, 1), p_OMC_O)]
    for i, significant_combination in enumerate(significant_combinations):
        # Columns corresponding to the datasets of interest
        x1 = significant_combination[0][0]
        x2 = significant_combination[0][1]
        # What level is this bar among the bars above the plot?
        level = len(significant_combinations) - i
        # Plot the bar
        bar_height = (y_range * 0.07 * level) + top
        bar_tips = bar_height - (y_range * 0.02)
        plt.plot(
            [x1, x1, x2, x2],
            [bar_tips, bar_height, bar_height, bar_tips], lw=1, c='k'
        )
        # Significance level
        p = significant_combination[1]
        if p < 0.001:
            sig_symbol = '***'
        elif p < 0.01:
            sig_symbol = '**'
        elif p < 0.05:
            sig_symbol = '*'
        else:
            sig_symbol = 'n.s.'
        text_height = bar_height + (y_range * 0.01)
        plt.text((x1 + x2) * 0.5, text_height, sig_symbol, ha='center', va='bottom', c='k')

    output = os.path.splitext(input)[0] + '_log10.png'
    plt.savefig(output)
    output = os.path.splitext(input)[0] + '_log10.svg'
    plt.savefig(output)
    plt.clf()

    # Boxplot with log10 abundance per sample
    ax = sns.boxplot(data=df, x='group', y='coverage', showfliers=False, palette=colors)
    sns.stripplot(data=df, x='group', y='coverage', color='black', size=4)
    plt.title(title)
    plt.xlabel('')
    plt.ylabel('Total coverage (%)')

    # Perform mannwhitneyu analysis
    # group_H = df[df['group'] == 'H']['reads']
    group_OMC = df[df['group'] == 'OMC']['reads']
    group_O = df[df['group'] == 'O']['reads']

    # Compare group H and OMC
    # u, p_H_OMC = mannwhitneyu(group_H, group_OMC)

    # Compare group H and O
    # u, p_H_O = mannwhitneyu(group_H, group_O)

    # Compare group OMC and O
    u, p_OMC_O = mannwhitneyu(group_OMC, group_O)

    # Get the y-axis limits
    bottom, top = ax.get_ylim()
    y_range = top - bottom

    # Significance bars
    # significant_combinations = [((0, 2), p_H_OMC), ((0, 1), p_H_O), ((1, 2), p_OMC_O)]
    significant_combinations = [((0, 1), p_OMC_O)]
    for i, significant_combination in enumerate(significant_combinations):
        # Columns corresponding to the datasets of interest
        x1 = significant_combination[0][0]
        x2 = significant_combination[0][1]
        # What level is this bar among the bars above the plot?
        level = len(significant_combinations) - i
        # Plot the bar
        bar_height = (y_range * 0.07 * level) + top
        bar_tips = bar_height - (y_range * 0.02)
        plt.plot(
            [x1, x1, x2, x2],
            [bar_tips, bar_height, bar_height, bar_tips], lw=1, c='k'
        )
        # Significance level
        p = significant_combination[1]
        if p < 0.001:
            sig_symbol = '***'
        elif p < 0.01:
            sig_symbol = '**'
        elif p < 0.05:
            sig_symbol = '*'
        else:
            sig_symbol = 'n.s.'
        text_height = bar_height + (y_range * 0.01)
        plt.text((x1 + x2) * 0.5, text_height, sig_symbol, ha='center', va='bottom', c='k')

    output = os.path.splitext(input)[0] + '_coverage.png'
    plt.savefig(output)
    output = os.path.splitext(input)[0] + '_coverage.svg'
    plt.savefig(output)
    plt.clf()

# Execution of the function with RNA reads
boxplot(input='./reads_aligned_per_sample_scaffold_11136_rna.tsv',
        title='Virus (scaffold 11136)\nRNA reads aligned per sample')

# Execution of the function with DNA VLPs reads
boxplot(input='./reads_aligned_per_sample_scaffold_11136_dna.tsv',
        title='Virus (scaffold 11136)\nDNA reads aligned per sample')




/var/folders/v4/vr3wk_q55dbbq68v6wdr2mrh0000gn/T/ipykernel_46374/3964677055.py:23: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.boxplot(data=df, x='group', y='reads', showfliers=False, palette=colors)
/var/folders/v4/vr3wk_q55dbbq68v6wdr2mrh0000gn/T/ipykernel_46374/3964677055.py:83: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.boxplot(data=df, x='group', y='relative_abundance', showfliers=False, palette=colors)
/var/folders/v4/vr3wk_q55dbbq68v6wdr2mrh0000gn/T/ipykernel_46374/3964677055.py:143: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.boxplot(d

<Figure size 640x480 with 0 Axes>